# pandas.DataFrame 로드

이 튜토리얼은 pandas 데이터 프레임을 tf.data.Dataset에 로드하는 방법의 예제를 제공합니다.

이 튜토리얼에서는 Cleveland Clinic Foundation for Heart Disease에서 제공하는 작은 데이터세트를 사용합니다. CSV에는 수백 개의 행이 있습니다. 각 행은 환자를 설명하고 각 열은 속성을 설명합니다. 이 정보를 사용하여 이 데이터세트에서 환자가 심장 질환이 있는지 여부를 예측하는 이진 분류 작업입니다.

## pandas를 사용하여 데이터 읽기

In [1]:
import pandas as pd
import tensorflow as tf

심장 데이터세트가 포함된 csv 파일을 다운로드합니다.

In [2]:
csv_file = tf.keras.utils.get_file('heart.csv', 'https://storage.googleapis.com/download.tensorflow.org/data/heart.csv')

16384/13273 [=====================================] - 0s 0us/step


pandas를 사용하여 csv 파일을 읽습니다.

In [3]:
df = pd.read_csv(csv_file)

In [4]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [5]:
df.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal         object
target        int64
dtype: object

데이터 프레임의 object thal 열을 이산 숫자 값으로 변환합니다.

In [6]:
df['thal'] = pd.Categorical(df['thal'])
df['thal'] = df.thal.cat.codes

In [7]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,2,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,4,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0


## tf.data.Dataset를 사용하여 데이터 로드하기

tf.data.Dataset.from_tensor_slices를 사용하여 pandas 데이터 프레임에서 값을 읽습니다.

tf.data.Dataset를 사용할 때의 이점 중 하나는 간단하고 효율적인 데이터 파이프라인을 작성할 수 있다는 것입니다. 자세한 내용은 데이터 로드 가이드를 참조하세요.

In [8]:
target = df.pop('target')

In [9]:
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))

In [10]:
for feat, targ in dataset.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

Features: [ 63.    1.    1.  145.  233.    1.    2.  150.    0.    2.3   3.    0.
   2. ], Target: 0
Features: [ 67.    1.    4.  160.  286.    0.    2.  108.    1.    1.5   2.    3.
   3. ], Target: 1
Features: [ 67.    1.    4.  120.  229.    0.    2.  129.    1.    2.6   2.    2.
   4. ], Target: 0
Features: [ 37.    1.    3.  130.  250.    0.    0.  187.    0.    3.5   3.    0.
   3. ], Target: 0
Features: [ 41.    0.    2.  130.  204.    0.    2.  172.    0.    1.4   1.    0.
   3. ], Target: 0


pd.Series는 __array__ 프로토콜을 구현하므로 np.array 또는 tf.Tensor를 사용하는 거의 모든 곳에서 투명하게 사용할 수 있습니다.

In [11]:
tf.constant(df['thal'])

<tf.Tensor: shape=(303,), dtype=int8, numpy=
array([2, 3, 4, 3, 3, 3, 3, 3, 4, 4, 2, 3, 2, 4, 4, 3, 4, 3, 3, 3, 3, 3,
       3, 4, 4, 3, 3, 3, 3, 4, 3, 4, 3, 4, 3, 3, 4, 2, 4, 3, 4, 3, 4, 4,
       2, 3, 3, 4, 3, 3, 4, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 4,
       4, 2, 3, 3, 4, 3, 4, 3, 3, 4, 4, 3, 3, 4, 4, 3, 3, 3, 3, 4, 4, 4,
       3, 3, 4, 3, 4, 4, 3, 4, 3, 3, 3, 4, 3, 4, 4, 3, 3, 4, 4, 4, 4, 4,
       3, 3, 3, 3, 4, 3, 4, 3, 4, 4, 3, 3, 2, 4, 4, 2, 3, 3, 4, 4, 3, 4,
       3, 3, 4, 2, 4, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4,
       4, 3, 3, 3, 4, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 3, 2,
       4, 4, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 2, 2, 4, 3, 4, 2, 4, 3,
       3, 4, 3, 3, 3, 3, 4, 3, 4, 3, 4, 2, 2, 4, 3, 4, 3, 2, 4, 3, 3, 2,
       4, 4, 4, 4, 3, 0, 3, 3, 3, 3, 1, 4, 3, 3, 3, 4, 3, 4, 3, 3, 3, 4,
       3, 3, 4, 4, 4, 4, 3, 3, 4, 3, 4, 3, 4, 4, 3, 4, 4, 3, 4, 4, 3, 3,
      

데이터세트를 섞고 배치 처리합니다.

In [12]:
train_dataset = dataset.shuffle(len(df)).batch(1)

## 모델 생성 및 훈련

In [13]:
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [14]:
model = get_compiled_model()
model.fit(train_dataset, epochs=15)

Epoch 1/15
303/303 [==============================] - 3s 6ms/step - loss: 6.5719 - accuracy: 0.6814
Epoch 2/15
303/303 [==============================] - 2s 6ms/step - loss: 0.9561 - accuracy: 0.6427
Epoch 3/15
303/303 [==============================] - 1s 5ms/step - loss: 0.7352 - accuracy: 0.7531
Epoch 4/15
303/303 [==============================] - 1s 4ms/step - loss: 0.8258 - accuracy: 0.6816
Epoch 5/15
303/303 [==============================] - 2s 6ms/step - loss: 0.5462 - accuracy: 0.7822
Epoch 6/15
303/303 [==============================] - 1s 4ms/step - loss: 0.5984 - accuracy: 0.7755
Epoch 7/15
303/303 [==============================] - 1s 4ms/step - loss: 0.7050 - accuracy: 0.7273
Epoch 8/15
303/303 [==============================] - 2s 6ms/step - loss: 0.5553 - accuracy: 0.7949
Epoch 9/15
303/303 [==============================] - 2s 6ms/step - loss: 0.5705 - accuracy: 0.7432
Epoch 10/15
303/303 [==============================] - 2s 6ms/step - loss: 0.4863 - accuracy: 0.7808

## 특성 열의 대안

사전을 모델에 대한 입력으로 전달하는 것은 tf.keras.layers.Input 레이어의 일치하는 사전을 작성하는 것만큼 간편하며, 함수형 API를 사용하여 필요한 사전 처리를 적용하고 레이어를 쌓습니다. 특성 열의 대안으로 사용할 수 있습니다.

In [15]:
inputs = {key: tf.keras.layers.Input(shape=(), name=key) for key in df.keys()}
x = tf.stack(list(inputs.values()), axis=-1)

x = tf.keras.layers.Dense(10, activation='relu')(x)
output = tf.keras.layers.Dense(1)(x)

model_func = tf.keras.Model(inputs=inputs, outputs=output)

model_func.compile(optimizer='adam',
                   loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=['accuracy'])

tf.data와 함께 사용했을 때 pd.DataFrame의 열 구조를 유지하는 가장 쉬운 방법은 pd.DataFrame을 dict로 변환하고 해당 사전을 조각화하는 것입니다.

In [16]:
dict_slices = tf.data.Dataset.from_tensor_slices((df.to_dict('list'), target.values)).batch(16)

In [17]:
for dict_slice in dict_slices.take(1):
  print (dict_slice)

({'age': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([63, 67, 67, 37, 41, 56, 62, 57, 63, 53, 57, 56, 56, 44, 52, 57])>, 'sex': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1])>, 'cp': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([1, 4, 4, 3, 2, 2, 4, 4, 4, 4, 4, 2, 3, 2, 3, 3])>, 'trestbps': <tf.Tensor: shape=(16,), dtype=int32, numpy=
array([145, 160, 120, 130, 130, 120, 140, 120, 130, 140, 140, 140, 130,
       120, 172, 150])>, 'chol': <tf.Tensor: shape=(16,), dtype=int32, numpy=
array([233, 286, 229, 250, 204, 236, 268, 354, 254, 203, 192, 294, 256,
       263, 199, 168])>, 'fbs': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0])>, 'restecg': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([2, 2, 2, 0, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 0, 0])>, 'thalach': <tf.Tensor: shape=(16,), dtype=int32, numpy=
array([150, 108, 129, 187, 172, 178, 160, 163, 147, 155, 148, 153, 142,
  

In [18]:
model_func.fit(dict_slices, epochs=15)

Epoch 1/15
19/19 [==============================] - 0s 9ms/step - loss: 9.3557 - accuracy: 0.7490
Epoch 2/15
19/19 [==============================] - 0s 9ms/step - loss: 2.6830 - accuracy: 0.5199
Epoch 3/15
19/19 [==============================] - 0s 9ms/step - loss: 2.0207 - accuracy: 0.6320
Epoch 4/15
19/19 [==============================] - 0s 7ms/step - loss: 1.7253 - accuracy: 0.5412
Epoch 5/15
19/19 [==============================] - 0s 6ms/step - loss: 1.1874 - accuracy: 0.6158
Epoch 6/15
19/19 [==============================] - 0s 8ms/step - loss: 0.8709 - accuracy: 0.6804
Epoch 7/15
19/19 [==============================] - 0s 10ms/step - loss: 0.6747 - accuracy: 0.7473
Epoch 8/15
19/19 [==============================] - 0s 6ms/step - loss: 0.5543 - accuracy: 0.7561
Epoch 9/15
19/19 [==============================] - 0s 8ms/step - loss: 0.4870 - accuracy: 0.7776
Epoch 10/15
19/19 [==============================] - 0s 8ms/step - loss: 0.4514 - accuracy: 0.7856
Epoch 11/15
19/19 